In [25]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import FMRegressor
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.ml import Pipeline

In [26]:
# Создание SparkSession
spark = SparkSession.builder.appName("CrabAgePrediction").getOrCreate()
# Загружаем датасет
data = spark.read.csv("CrabAgePrediction.csv", header = True, inferSchema = True)

data.printSchema()
data.show()

root
 |-- Sex: string (nullable = true)
 |-- Length: double (nullable = true)
 |-- Diameter: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Shucked Weight: double (nullable = true)
 |-- Viscera Weight: double (nullable = true)
 |-- Shell Weight: double (nullable = true)
 |-- Age: integer (nullable = true)

+---+------+--------+------+-----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|     Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+-----------+--------------+--------------+------------+---+
|  F|1.4375|   1.175|0.4125| 24.6357155|    12.3320325|     5.5848515|    6.747181|  9|
|  M|0.8875|    0.65|0.2125| 5.40057975|     2.2963095|    1.37495075|   1.5592225|  6|
|  I|1.0375|   0.775|  0.25| 7.95203475|      3.231843|    1.60174675|  2.76407625|  6|
|  F| 1.175|  0.8875|  0.25|13.48018725|    4.74854125|    2.28213475|   5.2446575| 10|
|  I|0.8875|  0.6

In [27]:
# Преобразуем Пол краба в числа
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
# Преобразовываем данные. Начинаем с выбора признаков
features = ["SexIndex", "Length", "Diameter" , "Height", "Weight", "Shucked Weight", "Viscera Weight", "Shell Weight"]
# Создаем вектор этих признаков
assembler = VectorAssembler(inputCols=features, outputCol="features")
# Разделяем данные на обучение и тест
(train, test) = data.randomSplit([0.7, 0.3])
# Создаем кортеж среднеквадратичных ошибок
tupleRMSE = ()
Pred_arr = []

train.show(3)
test.show(3)

print(train.count())
print(test.count())

+---+------+--------+------+---------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|   Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+---------+--------------+--------------+------------+---+
|  F|0.6875|  0.4875| 0.175|  2.26796|     0.8788345|    0.60951425|   0.7087375|  5|
|  F| 0.725|   0.525|0.1875|7.7961125|     3.2034935|    1.91359125|   0.9922325|  6|
|  F| 0.725|  0.5625|0.1875|  3.96893|    1.45999925|    0.66621325|     1.13398|  5|
+---+------+--------+------+---------+--------------+--------------+------------+---+
only showing top 3 rows

+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F|0.7625|  0.5625| 0.175|4.20990075|    1.65844575|    0.94970825|   1.2757275|  7|
|  F|0.7625|   0.575|   0

In [28]:
# Создаем модель
rf = RandomForestRegressor(featuresCol = "features", labelCol= "Age")
# Создаем конвейер
pipelineRF = Pipeline(stages = [indexer, assembler, rf])
# Обучение модели
modelRF = pipelineRF.fit(train)
# Предсказание возраста на тестовой выборке
predictionsRF = modelRF.transform(test).withColumnRenamed("prediction", "predictionsRF")
Pred_arr.append(predictionsRF)

In [29]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsRF", metricName = "rmse")
rmseRF = evaluator.evaluate(predictionsRF)
tupleRMSE += (rmseRF,)
print("Root Mean Squared Error (RMSE): %s" % rmseRF)
# Вывод результата и выключение SparkSession
predictionsRF.select("Age", "predictionsRF").show()

Root Mean Squared Error (RMSE): 2.2968902893194425
+---+-----------------+
|Age|    predictionsRF|
+---+-----------------+
|  7|6.547791450990421|
|  7| 6.73819352754214|
|  7|7.949678916288673|
|  6| 8.08743725417823|
| 10|8.058142570136978|
| 10|7.928545304210696|
| 10|8.598136234483665|
|  9| 8.85583305027954|
|  7|9.123932414360443|
| 10| 8.85583305027954|
| 10| 8.19249615377419|
|  8|8.893300837773735|
|  8|8.995222727383258|
| 11|8.900625376034906|
| 10|9.621921097809361|
| 12|9.621921097809361|
| 10|9.817364077523717|
| 10| 9.56225169006763|
|  8|9.623118280907953|
| 10|9.112820587873172|
+---+-----------------+
only showing top 20 rows



In [30]:
# Создание модели градиентного бустинга
gbt = GBTRegressor(featuresCol="features", labelCol= "Age")
# Создание конвейера
pipelineGBT = Pipeline(stages = [indexer, assembler, gbt])
# Обучение модели
modelGBT = pipelineGBT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGBT = modelGBT.transform(test).withColumnRenamed("prediction", "predictionsGBT")
Pred_arr.append(predictionsGBT)

In [31]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGBT", metricName = "rmse")
rmseGBT = evaluator.evaluate(predictionsGBT)
tupleRMSE += (rmseGBT,)
print("Root Mean Squared Error (RMSE): %s" % rmseGBT)
# Вывод результата и выключение SparkSession
predictionsGBT.select("Age", "predictionsGBT").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.383313050721814
+---+-----------------+
|Age|   predictionsGBT|
+---+-----------------+
|  7|6.075411154096466|
|  7|6.075411154096466|
|  7|6.971756362941675|
|  6|6.971756362941675|
| 10|6.971756362941675|
| 10|6.971756362941675|
| 10|9.500635255071424|
|  9|9.500635255071424|
|  7|9.081846499451178|
| 10|9.848389121102505|
| 10|6.919336152938989|
|  8|9.618236070268335|
|  8|9.618236070268335|
| 11|9.366529059603527|
| 10|9.681939074716167|
| 12|9.681939074716167|
| 10|9.681939074716167|
| 10|9.733154918936526|
|  8|9.681939074716167|
| 10|8.275004532605715|
+---+-----------------+
only showing top 20 rows



In [32]:
# Создание модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="Age", maxIter=10, regParam=0.3, elasticNetParam=0.8) 
# Создание конвейера
pipelineLR = Pipeline(stages = [indexer, assembler, lr])
# Fit the model
modelLR = pipelineLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsLR = modelLR.transform(test).withColumnRenamed("prediction", "predictionsLR")
Pred_arr.append(predictionsLR)

In [33]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsLR", metricName = "rmse")
rmseLR = evaluator.evaluate(predictionsLR)
tupleRMSE += (rmseLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseLR)
# Вывод результата и выключение SparkSession
predictionsLR.select("Age", "predictionsLR").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.587416345938417
+---+-----------------+
|Age|    predictionsLR|
+---+-----------------+
|  7|7.057776414693405|
|  7|7.269502919374844|
|  7|7.583789856443476|
|  6|7.522266442417324|
| 10|7.698112547033268|
| 10|7.194096770228231|
| 10|8.068766279832502|
|  9|8.046496124583914|
|  7|9.254000451489086|
| 10|8.157949053100811|
| 10| 7.84032718256324|
|  8|8.291206965470252|
|  8| 8.43985888224008|
| 11|8.175643516541687|
| 10|8.359606530344603|
| 12|8.359606530344603|
| 10|8.831373653281936|
| 10|8.929695111195803|
|  8| 8.66988707863793|
| 10|8.220454337980014|
+---+-----------------+
only showing top 20 rows



In [34]:
# Создание модели решающих деервьев
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineDT = Pipeline(stages = [indexer, assembler, dt])
# Fit the model
modelDT = pipelineDT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsDT = modelDT.transform(test).withColumnRenamed("prediction", "predictionsDT")
Pred_arr.append(predictionsDT)

In [35]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsDT", metricName = "rmse")
rmseDT = evaluator.evaluate(predictionsDT)
tupleRMSE += (rmseDT,)
print("Root Mean Squared Error (RMSE): %s" % rmseDT)
# Вывод результата и выключение SparkSession
predictionsDT.select("Age", "predictionsDT").show()

Root Mean Squared Error (RMSE): 2.431894521839743
+---+-----------------+
|Age|    predictionsDT|
+---+-----------------+
|  7|5.972972972972973|
|  7|5.972972972972973|
|  7|6.869318181818182|
|  6|6.869318181818182|
| 10|6.869318181818182|
| 10|6.869318181818182|
| 10|9.450617283950617|
|  9|9.450617283950617|
|  7|9.015873015873016|
| 10|9.450617283950617|
| 10|6.869318181818182|
|  8|9.450617283950617|
|  8|9.450617283950617|
| 11|9.450617283950617|
| 10|9.450617283950617|
| 12|9.450617283950617|
| 10|9.450617283950617|
| 10|9.450617283950617|
|  8|9.450617283950617|
| 10|6.869318181818182|
+---+-----------------+
only showing top 20 rows



In [36]:
# Создание модели изотонической регрессии
ir = IsotonicRegression(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineIR = Pipeline(stages = [indexer, assembler, ir])
# Fit the model
modelIR = pipelineIR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsIR = modelIR.transform(test).withColumnRenamed("prediction", "predictionsIR")
Pred_arr.append(predictionsIR)

In [37]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsIR", metricName = "rmse")
rmseIR = evaluator.evaluate(predictionsIR)
tupleRMSE += (rmseIR,)
print("Root Mean Squared Error (RMSE): %s" % rmseIR)
# Вывод результата и выключение SparkSession
predictionsIR.select("Age", "predictionsIR").show()

Root Mean Squared Error (RMSE): 3.3115332238963915
+---+-------------+
|Age|predictionsIR|
+---+-------------+
|  7|         12.0|
|  7|         12.0|
|  7|         12.0|
|  6|         12.0|
| 10|         12.0|
| 10|         12.0|
| 10|         12.0|
|  9|         12.0|
|  7|         12.0|
| 10|         12.0|
| 10|         12.0|
|  8|         12.0|
|  8|         12.0|
| 11|         12.0|
| 10|         12.0|
| 12|         12.0|
| 10|         12.0|
| 10|         12.0|
|  8|         12.0|
| 10|         12.0|
+---+-------------+
only showing top 20 rows



In [38]:
# Создание модели FMRegressor
fm = FMRegressor(featuresCol="features", labelCol="Age", predictionCol="prediction",stepSize=0.001)
# Создание конвейера
pipelineFM = Pipeline(stages = [indexer, assembler, fm])
# Fit the model
modelFM = pipelineFM.fit(train)
# Предсказание возраста на тестовой выборке
predictionsFM = modelFM.transform(test).withColumnRenamed("prediction", "predictionsFM")
Pred_arr.append(predictionsFM)

In [39]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsFM", metricName = "rmse")
rmseFM = evaluator.evaluate(predictionsFM)
tupleRMSE += (rmseFM,)
print("Root Mean Squared Error (RMSE): %s" % rmseFM)
# Вывод результата и выключение SparkSession
predictionsFM.select("Age", "predictionsFM").show()

Root Mean Squared Error (RMSE): 5.501177297319231
+---+------------------+
|Age|     predictionsFM|
+---+------------------+
|  7|1.1558124882866843|
|  7| 1.219434831249258|
|  7|1.4506220074305543|
|  6| 1.521635200204903|
| 10| 1.566381007957242|
| 10| 1.533985190053405|
| 10| 1.728594189162549|
|  9|1.9705759190769276|
|  7| 4.312132920243425|
| 10|2.1028033964125905|
| 10|1.7282660259821798|
|  8| 2.589498793456741|
|  8| 2.286970190341163|
| 11| 2.618234916975305|
| 10| 2.691628504820664|
| 12| 2.758616565151602|
| 10| 2.753415598634043|
| 10|3.4117148504135986|
|  8|2.9637922944265247|
| 10|2.4103784922167746|
+---+------------------+
only showing top 20 rows



In [40]:
# Модель  `AFTSurvivalRegression`  не подходит для датасета  `CrabAgePrediction`, так как нет информации о том,  
# живы ли были крабы, когда их поймали.

# Эта модель предназначена для анализа данных выживания, где есть данные о времени до события (в данном случае, смерть) 
# и информация о том,  было ли это событие наблюдено (краб умер) или  **цензурировано**  (краб был жив, когда его поймали).  

In [41]:
glr = GeneralizedLinearRegression(featuresCol="features", labelCol="Age", family="gaussian", link="identity", maxIter=10, regParam=0.3)
# Создание конвейера
pipelineGLR = Pipeline(stages = [indexer, assembler, glr])
# Fit the model
modelGLR = pipelineGLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGLR = modelGLR.transform(test).withColumnRenamed("prediction", "predictionsGLR")
Pred_arr.append(predictionsGLR)

In [42]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGLR", metricName = "rmse")
rmseGLR = evaluator.evaluate(predictionsGLR)
tupleRMSE += (rmseGLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseGLR)
# Вывод результата и выключение SparkSession
predictionsGLR.select("Age", "predictionsGLR").show()

Root Mean Squared Error (RMSE): 2.4062932534980455
+---+------------------+
|Age|    predictionsGLR|
+---+------------------+
|  7|6.6594744843741545|
|  7| 6.879171833632027|
|  7| 7.352699244217624|
|  6| 7.271523204683127|
| 10| 7.594577460195217|
| 10| 7.089484306472986|
| 10| 8.056355277679913|
|  9| 7.967596736937042|
|  7| 8.596027301132233|
| 10|  8.19711644108072|
| 10|   8.0004468172759|
|  8| 8.312250816583543|
|  8| 8.579414216181283|
| 11| 8.004434570814318|
| 10| 8.398345478339014|
| 12| 8.413784583035078|
| 10|  9.08310266875322|
| 10| 8.996540757509479|
|  8| 8.832569898073547|
| 10| 8.555915644254046|
+---+------------------+
only showing top 20 rows



In [43]:
# Создаем списки моделей, выполняющихся том же в порядке выше
modelsList = ["RandomForestRegressor", "GBTRegressor", "LinearRegression", "DecisionTreeRegressor", 
              "IsotonicRegression", "FMRegressor", "GeneralizedLinearRegression"]
shortModelsList = ["predictionsRF", "predictionsGBT", "predictionsLR", "predictionsDT", "predictionsIR", 
                   "predictionsFM", "predictionsGLR"]

# Схема нового DataFrame значений RMSE каждой модели
schemaRMSE = StructType([
    StructField(modelsList[0], DoubleType(), True),
    StructField(modelsList[1], DoubleType(), True),
    StructField(modelsList[2], DoubleType(), True),
    StructField(modelsList[3], DoubleType(), True),
    StructField(modelsList[4], DoubleType(), True),
    StructField(modelsList[5], DoubleType(), True),
    StructField(modelsList[6], DoubleType(), True)
])
# Создание DataFrame
rmseDF = spark.createDataFrame([tupleRMSE], schemaRMSE)

In [46]:
for i, df in enumerate(Pred_arr):
    df = df.withColumn("id", monotonically_increasing_id())
    Pred_arr[i] = df

ListSelectedDF = [df.select("Age", "id", shortModelsList[i]) for i, df in enumerate(Pred_arr)]
predDF = ListSelectedDF[0]

for i in range(1, len(ListSelectedDF)):
    predDF = predDF.join(ListSelectedDF[i], on = ["Age", "id"], how = "inner")

predDF = predDF.drop("id")


In [47]:
# Вывод лучшего результата
print(f"Лучшая модель: {modelsList[rmseDF.head().index(min(rmseDF.head()))]} (RMSE: {min(rmseDF.head())})")
# Запись результатов в csv-файл
rmseDF.write.csv("RMSE.csv", mode = "overwrite", header = True)
predDF.write.csv("Pred.csv", mode = "overwrite", header = True)

Лучшая модель: RandomForestRegressor (RMSE: 2.2968902893194425)


In [ ]:
spark.stop()